In [1]:
import pandas as pd
import numpy as np
import sys, os , time, pickle
from tabulate import tabulate
from IPython.display import display

methods_list_aux = {'hardMV':'hard-MV', 'softMV': 'soft-MV', 'DS':'D&S', "Raykar": "Raykar", 
                   'OursGlobal': 'Ours Global','OursIndividualK': 'Ours Individual', 'OursIndividualT':'Ours Individual' }

    
def find_match(string,list_des):
    return [value for value in list_des if string in value]

def plot_files(file_name):
    print("\nFile to show: ",file_name)
    table = pd.read_csv(file_name)
    display(table)
    #print(tabulate(table, headers='keys', tablefmt='rst')) #latex
    
def calculate_aux(archivo,what_to_do="mv",model=""):
    try:
        f = open(archivo)
        result_list = []
        for line in f.readlines():
            if what_to_do == "mv":
                if "ACC MV" in line: 
                    result_list.append(float(line.split(" ")[-1].strip()))
            elif what_to_do == 'epochs':
                if "Epochs to converge" in line and model in line: 
                    result_list.append(float(line.split(" ")[-1].strip()))
        f.close()
        return result_list
    except:
        return "-"
    
def compare_table(tipo,repre="normal", std=False):
    t = pd.DataFrame()
    t["Methods"] = ["Ann(tr)"]+methods_list
    t["Train"] = ["-"] + ["-" for _ in range(len(methods_list))]
    t["Test"] = ["-"] + ["-" for _ in range(len(methods_list))]
    if tipo == 'acc':
        try:
            t.loc[0,"Train"] =  round(np.mean(calculate_aux(folder+"results.txt",what_to_do="mv"))*100,2)
        except:
            try:
                t.loc[0,"Train"] =  round(np.mean(calculate_aux(folder+"results_ind.txt",what_to_do="mv"))*100,2)
            except:
                print("No se encontró acc")
            
    for k, method in enumerate(methods_list):
        aux = find_match(method,files)
        if tipo == "acc" or tipo=="f1": #como normal?
            if tipo=="acc":
                to_extract = "Accuracy"
            elif tipo == "f1":
                to_extract = "F1 (macro)"
            if std:
                test_aux = find_match("_train_std.csv",aux)
                t.loc[k+1,"Train"] = round(pd.read_csv(folder+test_aux[-1])[to_extract][0]*100, 2)
                train_aux = find_match("_test_std.csv",aux)
                t.loc[k+1,"Test"] = round(pd.read_csv(folder+train_aux[-1])[to_extract][0]*100, 2)
            else:
                test_aux = find_match("_train.csv",aux)
                t.loc[k+1,"Train"] = round(pd.read_csv(folder+test_aux[-1])[to_extract][0]*100, 2)
                train_aux = find_match("_test.csv",aux)
                t.loc[k+1,"Test"] = round(pd.read_csv(folder+train_aux[-1])[to_extract][0]*100, 2)

        elif "norm" in tipo.lower() or "js" in tipo.lower():
            if std:
                kl_aux = find_match("_train_std.csv",aux)
            else:
                kl_aux = find_match("_train.csv",aux)
            
            if len(kl_aux) != 0:                
                if repre=="R":
                    to_extract = "("+repre+") "+tipo+ " mean"
                elif repre=='G':
                    to_extract = "("+repre+") "+tipo
                try:
                    t.loc[k+1,"Train"] = round(pd.read_csv(folder+kl_aux[-1])[to_extract][0] ,3)
                except:
                    pass
        elif tipo == "epochs":
            t.loc[k+1,"Train"]=np.mean(calculate_aux(folder+"results.txt",what_to_do="epochs",model=methods_list_aux[method]))
    return t

def get_value(lista, name, std=False):
    i = 0
    if std:
        i = 1
    return np.asarray([value[i][name][0] if type(value[i])== pd.DataFrame else -9999 for value in lista] )

def create_df(method_l,T_try, value, file_re = "./results.txt",digits=4):
    t = pd.DataFrame(np.zeros((len(method_l)+1,len(T_try))))    
    t.columns = T_try
    t.index = ["Ann(tr)"]+method_l
    t_std = t.copy()
    
    if value.lower() == "accuracy" or "F1" in value.lower():
        if len(calculate_aux(file_re,what_to_do="mv")) > len(T_try) +1:
            aux2 = calculate_aux(file_re,what_to_do="mv")
            aux = [np.mean(aux2[i:30*(i+1)]) for i in range(0,len(aux2),30)]
            aux2 = [np.std(aux2[i:30*(i+1)]) for i in range(0,len(aux2),30)]
        else:
            aux = calculate_aux(file_re,what_to_do="mv")
            aux2 = np.zeros((len(T_try)))
        t.loc["Ann(tr)",:] = aux
        t_std.loc["Ann(tr)",:] = aux2
        for method_l in methods_list:
            t.loc[method_l,:] = get_value(dic_res[method_l]["test"], value)
            t_std.loc[method_l,:] = get_value(dic_res[method_l]["test"], value, std=True)

    elif "inertia" in value.lower() or "iner" in value.lower():
        for method_l in methods_list:
            if "Ours" in method_l:
                t.loc[method_l,:] = get_value(dic_res[method_l]["testAux"], value)
                
    else:
        t.index = ["D_t"]+method_l
        t.loc["D_t",:] = D_t
        for method_l in methods_list:
            t.loc[method_l,:] = get_value(dic_res[method_l]["train"], value)
            t_std.loc[method_l,:] = get_value(dic_res[method_l]["train"], value, std=True)
    return t.round(digits), t_std.round(digits)

!ls

CIFAR_T		Fashion_Q4	LabelMe		    Sentiment_Tol3
Fashion_Q1_0.6	Fashion_Q4_0.6	LabelMe_v2	    Sentiment_Tol5
Fashion_Q3_0.7	Fashion_Q6_0.5	Plot_Results.ipynb  simple_T


In [13]:
!ls Fashion_Q4_0.6/
folder = "Fashion_Q4_0.6/"

files = sorted(os.listdir(folder))
methods_list = list(set([f.split("_")[1] for f in os.listdir(folder) if (len(f.split("_")) >2) and ("UpperBound" not in f)]))
methods_list

Fashion_DS_test.csv
Fashion_DS_test_std.csv
Fashion_DS_train.csv
Fashion_DS_train_std.csv
Fashion_hardMV_test.csv
Fashion_hardMV_test_std.csv
Fashion_hardMV_train.csv
Fashion_hardMV_train_std.csv
Fashion_OursGlobal_testAux.csv
Fashion_OursGlobal_testAux_std.csv
Fashion_OursGlobal_test.csv
Fashion_OursGlobal_test_std.csv
Fashion_OursGlobal_train.csv
Fashion_OursGlobal_train_std.csv
Fashion_OursIndividualK_testAux.csv
Fashion_OursIndividualK_testAux_std.csv
Fashion_OursIndividualK_test.csv
Fashion_OursIndividualK_test_std.csv
Fashion_OursIndividualK_train.csv
Fashion_OursIndividualK_train_std.csv
Fashion_OursIndividualT_testAux.csv
Fashion_OursIndividualT_testAux_std.csv
Fashion_OursIndividualT_test.csv
Fashion_OursIndividualT_test_std.csv
Fashion_OursIndividualT_train.csv
Fashion_OursIndividualT_train_std.csv
Fashion_Raykar_test.csv
Fashion_Raykar_test_std.csv
Fashion_Raykar_train.csv
Fashion_Raykar_train_std.csv
Fashion_softMV_test.csv
Fashion_softMV_test_std.csv
Fashion_softMV_train.c

['OursIndividualT',
 'OursIndividualK',
 'softMV',
 'OursGlobal',
 'DS',
 'Raykar',
 'hardMV']

In [14]:
t_acc = compare_table("acc")
t_acc_std = compare_table("acc", std=True)

t_norm_R = compare_table("NormF","R")
t_norm_R_std = compare_table("NormF","R", std=True)
t_js_R = compare_table("JS","R")
t_js_R_std = compare_table("JS","R",  std=True)

t_norm_G = compare_table("NormF","G")
t_norm_G_std = compare_table("NormF","G", std=True)
t_js_G = compare_table("JS","G")
t_js_G_std = compare_table("JS","G", std=True)

t_epochs = compare_table("epochs")

In [9]:
## Q4 -- 0.6 (test es los que fueron eliminados)
from IPython.display import display
display(t_acc.merge(t_acc_std.iloc[:,1:], left_index=True, right_index=True))
t_acc = compare_table("f1")
t_acc_std = compare_table("f1", std=True)
display(t_acc.merge(t_acc_std.iloc[:,1:], left_index=True, right_index=True))

,Methods,Train_x,Test_x,Train_y,Test_y
0,Ann(tr),60.19,-,60.19,-
1,OursIndividualT,66.24,76.63,0.26,0.83
2,OursIndividualK,65.1,75.26,0.55,1.01
3,softMV,62.31,75.93,0.34,0.56
4,OursGlobal,65.22,77.45,0.4,0.85
5,DS,62.4,76.49,0.41,0.38
6,Raykar,64.38,78.45,0.21,0.51
7,hardMV,60.11,74.14,0.34,0.45


,Methods,Train_x,Test_x,Train_y,Test_y
0,Ann(tr),-,-,-,-
1,OursIndividualT,65.26,73.38,0.31,0.78
2,OursIndividualK,62.88,70.16,1.31,1.37
3,softMV,60.48,70.92,0.84,0.96
4,OursGlobal,63.56,73.06,0.72,1.29
5,DS,58.16,69.15,1.13,1.13
6,Raykar,62.76,74.37,0.69,0.62
7,hardMV,51.02,60.66,0.84,1.01


In [24]:
## Q4 -- 0.5 (test es los que fueron eliminados + otros de train)
from IPython.display import display
display(t_acc.merge(t_acc_std.iloc[:,1:], left_index=True, right_index=True))
t_acc = compare_table("f1")
t_acc_std = compare_table("f1", std=True)
display(t_acc.merge(t_acc_std.iloc[:,1:], left_index=True, right_index=True))

,Methods,Train_x,Test_x,Train_y,Test_y
0,Ann(tr),0.686195,-,0.686195,-
1,OursGlobal,0.6777,0.6532,0.0044,0.0055
2,Raykar,0.6551,0.662,0.002,0.0022
3,hardMV,0.6219,0.6221,0.0021,0.0042
4,softMV,0.6402,0.6463,0.0021,0.0045
5,OursIndividualT,0.6979,0.6439,0.003,0.0035
6,DS,0.6364,0.6399,0.0023,0.004
7,OursIndividualK,0.6976,0.638,0.0031,0.0069


,Methods,Train_x,Test_x,Train_y,Test_y
0,Ann(tr),-,-,-,-
1,OursGlobal,0.6602,0.6295,0.0079,0.0101
2,Raykar,0.64,0.6436,0.0058,0.0064
3,hardMV,0.5406,0.5329,0.0064,0.0102
4,softMV,0.6213,0.6228,0.0064,0.0094
5,OursIndividualT,0.6867,0.6269,0.0036,0.0028
6,DS,0.5951,0.5942,0.0075,0.0099
7,OursIndividualK,0.6822,0.6166,0.0049,0.0077


In [27]:
## sentiment.. (tol mas alta para mio - 5)
from IPython.display import display
display(t_acc.merge(t_acc_std.iloc[:,1:], left_index=True, right_index=True))
t_acc = compare_table("f1")
t_acc_std = compare_table("f1", std=True)
display(t_acc.merge(t_acc_std.iloc[:,1:], left_index=True, right_index=True))

,Methods,Train_x,Test_x,Train_y,Test_y
0,Ann(tr),0.889578,-,0.889578,-
1,OursGlobal,0.8821,0.7573,0.0038,0.0053
2,Raykar,0.8297,0.7374,0.0124,0.0154
3,hardMV,0.8686,0.7422,0.0167,0.0117
4,softMV,0.7927,0.7443,0.0167,0.0084
5,OursIndividualT,0.8979,0.7634,0.0026,0.0072
6,DS,0.8807,0.7521,0.0435,0.0119
7,OursIndividualK,0.8795,0.7529,0.0055,0.0065


,Methods,Train_x,Test_x,Train_y,Test_y
0,Ann(tr),-,-,-,-
1,OursGlobal,0.882,0.7568,0.0039,0.0056
2,Raykar,0.8293,0.7356,0.0127,0.019
3,hardMV,0.8685,0.7413,0.0187,0.0128
4,softMV,0.7915,0.7431,0.0187,0.0099
5,OursIndividualT,0.8979,0.763,0.0026,0.0076
6,DS,0.8802,0.7513,0.0443,0.0131
7,OursIndividualK,0.8793,0.7523,0.0056,0.0069


In [10]:
t_epochs

,Methods,Train,Test
0,Ann(tr),-,-
1,OursIndividualT,24.8163,-
2,OursIndividualK,24.8163,-
3,softMV,4,-
4,OursGlobal,13.5,-
5,DS,4,-
6,Raykar,5.0425,-
7,hardMV,4,-


In [18]:
#t_acc = compare_table("f1")
#t_acc_std = compare_table("f1", std=True)
#aux = t_acc.merge(t_acc_std.iloc[:,1:], left_index=True, right_index=True)

#aux = t_js_R.merge(t_js_R_std.iloc[:,1:], left_index=True, right_index=True)
aux = t_js_G.merge(t_js_G_std.iloc[:,1:], left_index=True, right_index=True)

aux_str = tabulate(aux,headers='keys',tablefmt='latex').split("\n")[2:-2]
columns = [a.replace('\\' , '').strip() for a in aux_str[0].split("&")[1:]]

text = "" 
for value in aux_str[2:]:
    split_v = [a.replace('\\' , '').strip() for a in value.split("&")]
    text += split_v[1].strip() + " & "
    for i, a in enumerate(split_v[1:]):
        if "x" in columns[i]:
            for j, b in enumerate(split_v[1:]):
                if columns[i].replace("x","y") == columns[j]:
                    text += "$"+ str(a) + " \pm " + str(b)+ "$"
                    if columns[j] != columns[-1]: #si no es el ultimo
                        text +=   " & "   
    text += " \\\ \n"
print(text)

Ann(tr) & $- \pm -$ & $- \pm -$ \\ 
OursIndividualT & $0.086 \pm 0.005$ & $- \pm -$ \\ 
OursIndividualK & $0.109 \pm 0.018$ & $- \pm -$ \\ 
softMV & $0.016 \pm 0.0$ & $- \pm -$ \\ 
OursGlobal & $0.101 \pm 0.007$ & $- \pm -$ \\ 
DS & $0.022 \pm 0.0$ & $- \pm -$ \\ 
Raykar & $0.058 \pm 0.0$ & $- \pm -$ \\ 
hardMV & $0.047 \pm 0.0$ & $- \pm -$ \\ 



## T change

In [5]:
folder = "./simple_T//"

In [6]:
to_check = [100,500,1500,3500,6000,10000]

if 'simple' in folder.lower():
    T_data = 5 #for now is this..
    N = 2500
elif "cifar" in folder.lower():
    T_data = 3
    N = 50000
D_t = np.asarray([N*T_data/T for T in to_check])

files_f = [f for f in os.listdir(folder) if ".pickle" in f]
dic_res = {}
for file in files_f:
    #if len(file.split("_")) >2:
    _, method_name, method_type = (file.split(".pickle")[0]).split("_")
    with open(folder+file, 'rb') as handle:
        if method_name in dic_res:
            dic_res[method_name][method_type] = pickle.load(handle)
        else:
            dic_res[method_name] = {}
            dic_res[method_name][method_type] = pickle.load(handle)
methods_list = list(dic_res.keys())
methods_list

['hardMV',
 'DS',
 'OursGlobal',
 'OursIndividualT',
 'softMV',
 'OursIndividualK',
 'Raykar']

In [42]:
t, t_std = create_df(methods_list, to_check,"Accuracy", file_re=folder+'results.txt')
t.merge(t_std, left_index=True, right_index=True)

,100_x,500_x,1500_x,3500_x,6000_x,10000_x,100_y,500_y,1500_y,3500_y,6000_y,10000_y
Ann(tr),0.7211,0.7201,0.7197,0.7196,0.7210,0.7229,0.0257,0.0103,0.0100,0.0098,0.0093,0.0075
hardMV,0.7939,0.8046,0.7795,0.7692,0.7825,0.8065,0.0771,0.0515,0.0689,0.0696,0.0699,0.0480
DS,0.9403,0.9350,0.9289,0.9184,0.8923,0.8598,0.0069,0.0100,0.0130,0.0202,0.0386,0.0690
OursGlobal,0.9133,0.9093,0.8941,0.8891,0.9071,0.9122,0.0119,0.0193,0.1022,0.1010,0.0158,0.0168
OursIndividualT,0.9416,0.9405,0.9171,0.9256,0.9172,0.8479,0.0061,0.0073,0.0137,0.0271,0.0436,0.0879
softMV,0.6825,0.6823,0.6786,0.6619,0.6657,0.6771,0.0485,0.0344,0.0518,0.0244,0.0307,0.0299
OursIndividualK,0.9418,0.9459,0.9430,0.9172,0.8775,0.7960,0.0071,0.0041,0.0080,0.0356,0.0719,0.1216
Raykar,0.9405,0.9349,0.9258,0.8435,0.7597,0.7156,0.0048,0.0076,0.0123,0.1267,0.1223,0.1044


In [91]:
t, t_std = create_df(methods_list,to_check, "(R) JS mean", digits=3)
#t, t_std = create_df(methods_list,to_check, "(R) NormF mean")
t.merge(t_std, left_index=True, right_index=True)

,100_x,500_x,1500_x,3500_x,6000_x,10000_x,100_y,500_y,1500_y,3500_y,6000_y,10000_y
hardMV,0.087,0.086,0.087,0.088,0.087,0.087,0.008,0.003,0.003,0.001,0.001,0.001
DS,0.012,0.067,0.177,0.252,0.280,0.305,0.001,0.002,0.003,0.002,0.003,0.002
OursGlobal,0.106,0.109,0.114,0.118,0.112,0.111,0.018,0.017,0.037,0.030,0.016,0.016
OursIndividualT,0.018,0.022,0.054,0.113,0.151,0.193,0.003,0.003,0.007,0.009,0.008,0.019
softMV,0.087,0.086,0.087,0.087,0.087,0.087,0.005,0.003,0.002,0.001,0.001,0.001
OursIndividualK,0.020,0.021,0.047,0.092,0.139,0.192,0.009,0.004,0.005,0.005,0.010,0.016
Raykar,0.014,0.056,0.114,0.157,0.173,0.182,0.001,0.001,0.001,0.005,0.003,0.003


In [93]:
t, t_std = create_df(methods_list,to_check,"(G) JS", digits=3)
#t, t_std = create_df(methods_list,to_check,"(G) NormF")
t.merge(t_std, left_index=True, right_index=True)

,100_x,500_x,1500_x,3500_x,6000_x,10000_x,100_y,500_y,1500_y,3500_y,6000_y,10000_y
hardMV,0.017,0.017,0.017,0.018,0.017,0.017,0.004,0.001,0.002,0.002,0.001,0.001
DS,0.000,0.002,0.005,0.022,0.038,0.052,0.000,0.001,0.001,0.001,0.002,0.002
OursGlobal,0.049,0.051,0.055,0.058,0.053,0.052,0.012,0.018,0.037,0.029,0.016,0.018
OursIndividualT,0.004,0.003,0.009,0.005,0.011,0.031,0.002,0.002,0.004,0.003,0.003,0.016
softMV,0.014,0.014,0.014,0.014,0.014,0.014,0.001,0.001,0.001,0.001,0.000,0.001
OursIndividualK,0.007,0.003,0.003,0.009,0.018,0.034,0.004,0.002,0.001,0.004,0.006,0.011
Raykar,0.002,0.021,0.060,0.092,0.105,0.113,0.000,0.001,0.002,0.004,0.003,0.003


In [ ]:
aux = t.merge(t_std, left_index=True, right_index=True)*100
print(tabulate(aux,headers='keys',tablefmt='latex'))

In [94]:
aux = t.merge(t_std, left_index=True, right_index=True) #*100 solo acc
aux_str = tabulate(aux,headers='keys',tablefmt='latex').split("\n")[2:-2]
columns = [a.replace('\\' , '').strip() for a in aux_str[0].split("&")[1:]]

text = "" 
for value in aux_str[2:]:
    split_v = [a.replace('\\' , '').strip() for a in value.split("&")]
    text += split_v[0].strip() + " & "
    for i, a in enumerate(split_v[1:]):
        if "x" in columns[i]:
            for j, b in enumerate(split_v[1:]):
                if columns[i].replace("x","y") == columns[j]:
                    text += "$"+ str(a) + " \pm " + str(b)+ "$"
                    if columns[j] != columns[-1]: #si no es el ultimo
                        text +=   " & "   
    text += " \\\ \n"
print(text)

hardMV & $0.017 \pm 0.004$ & $0.017 \pm 0.001$ & $0.017 \pm 0.002$ & $0.018 \pm 0.002$ & $0.017 \pm 0.001$ & $0.017 \pm 0.001$ \\ 
DS & $0 \pm 0$ & $0.002 \pm 0.001$ & $0.005 \pm 0.001$ & $0.022 \pm 0.001$ & $0.038 \pm 0.002$ & $0.052 \pm 0.002$ \\ 
OursGlobal & $0.049 \pm 0.012$ & $0.051 \pm 0.018$ & $0.055 \pm 0.037$ & $0.058 \pm 0.029$ & $0.053 \pm 0.016$ & $0.052 \pm 0.018$ \\ 
OursIndividualT & $0.004 \pm 0.002$ & $0.003 \pm 0.002$ & $0.009 \pm 0.004$ & $0.005 \pm 0.003$ & $0.011 \pm 0.003$ & $0.031 \pm 0.016$ \\ 
softMV & $0.014 \pm 0.001$ & $0.014 \pm 0.001$ & $0.014 \pm 0.001$ & $0.014 \pm 0.001$ & $0.014 \pm 0$ & $0.014 \pm 0.001$ \\ 
OursIndividualK & $0.007 \pm 0.004$ & $0.003 \pm 0.002$ & $0.003 \pm 0.001$ & $0.009 \pm 0.004$ & $0.018 \pm 0.006$ & $0.034 \pm 0.011$ \\ 
Raykar & $0.002 \pm 0$ & $0.021 \pm 0.001$ & $0.06 \pm 0.002$ & $0.092 \pm 0.004$ & $0.105 \pm 0.003$ & $0.113 \pm 0.003$ \\ 



In [86]:
t, _ = create_df(methods_list,to_check,"Iner JS")
#t, _ = create_df(methods_list,to_check,"Iner NormF")
t.merge(t_std, left_index=True, right_index=True)

,100_x,500_x,1500_x,3500_x,6000_x,10000_x,100_y,500_y,1500_y,3500_y,6000_y,10000_y
Ann(tr),0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
hardMV,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0038,0.0014,0.0017,0.0016,0.0014,0.0013
DS,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0002,0.0006,0.0010,0.0013,0.0016,0.0017
OursGlobal,0.2384,0.2216,0.2078,0.2109,0.2151,0.2314,0.0121,0.0181,0.0370,0.0287,0.0164,0.0175
OursIndividualT,0.3282,0.3454,0.4181,0.6120,0.6656,0.7231,0.0016,0.0019,0.0045,0.0027,0.0025,0.0157
softMV,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0010,0.0008,0.0009,0.0008,0.0005,0.0006
OursIndividualK,0.3002,0.3366,0.3954,0.5000,0.5884,0.6733,0.0039,0.0019,0.0012,0.0036,0.0064,0.0110
Raykar,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0003,0.0008,0.0018,0.0038,0.0027,0.0029
